In [25]:
import os
import re

# wget --page-requisites --mirror http://localhost:32771
# cp -rf ./localhost\:32771/* /mnt/c/repos/bc-static/

In [26]:
top = "C:\\repos\\bc-static"
exclude = [".git"]
global walk_results
walk_results = []
for root, dirs, files in os.walk(top, topdown=True):
    dirs[:] = [d for d in dirs if d not in exclude]
    files[:] = [f for f in files if f.endswith('index.html')]
    walk_results.append({'root':root,'dirs':dirs,'files':files})
    
print("Files found: " + str(len(walk_results)))

Files found: 123


In [27]:
feed_rename_count = 0
feed_found_count = 0
for result in walk_results:
    if result['root'].endswith("feed"):
        feed_found_count = feed_found_count + 1
        if len(result['files']) is not 0:
            # Pre-existing rss.xml file might be there, delete it
            if os.path.isfile(result['root']+'\\'+'rss.xml'):
                os.remove(result['root']+'\\'+'rss.xml')
            os.rename(result['root']+'\\'+result['files'][0], result['root']+'\\'+'rss.xml')
            result['files'][0] = 'rss.xml'
            feed_rename_count = feed_rename_count + 1
print("Feed files found: " + str(feed_found_count))
print("Feed files renamed: " + str(feed_rename_count))

Feed files found: 33
Feed files renamed: 32


In [28]:
global full_paths
full_paths = []
for result in walk_results:
    for file in result.get('files'):
        full_path = os.path.join(result.get('root'), file)
        full_paths.append(full_path)

print("Full paths found: " + str(len(full_paths)))

Full paths found: 68


In [29]:
local_url = 'http://localhost:32771'
public_url = 'https://brilliantcoding.com'
local_url_json = local_url.replace('/','\\\/')
public_url_json = public_url.replace('/','\\\/')
html_pattern =re.compile('(?i){0}'.format(local_url))
json_pattern = re.compile('(?i){0}'.format(local_url_json))
print(html_pattern.pattern + '->' + public_url)
print(json_pattern.pattern + '->' + public_url_json)

original_feed_path = '/feed/(?!rss)'
rss_feed_path = '/feed/rss.xml'
feed_pattern = re.compile('(?i){0}'.format(original_feed_path))
print(feed_pattern.pattern + '->' + rss_feed_path)

i = 0
for full_path in full_paths:
    modified = False
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = list(f)
        out_lines = []
        ii = 0
        for line in lines:
            html_match = html_pattern.search(line)
            if (html_match != None):
                modified = True
                before_line = line
                count = len(html_pattern.findall(line))
                line = html_pattern.sub(public_url,line) 
                if i < 10 and ii <10:
                    print(before_line)
                    print(line)
                    print('HTML Replacements:{0}'.format(count))
            json_match = json_pattern.search(line)
            if (json_match != None):
                modified = True
                before_line = line
                count = len(json_pattern.findall(line))
                line = json_pattern.sub(public_url_json,line)
                if i < 10 and ii <10:
                    print(before_line)
                    print(line)
                    print('Json Replacements:{0}'.format(count))
            feed_match = feed_pattern.search(line)
            if (feed_match != None):
                modified = True
                before_line = line
                count = len(feed_pattern.findall(line))
                line = feed_pattern.sub(rss_feed_path,line)
                if i < 10 and ii <10:
                    print(before_line)
                    print(line)
                    print('Feed Replacements:{0}'.format(count))
                    
            out_lines.append(line)

            ii = ii + 1
        if modified:    
            with open(full_path, 'w', encoding='utf-8') as ff:
                ff.writelines(out_lines)
        i = i + 1
print('success')

(?i)http://localhost:32771->https://brilliantcoding.com
(?i)http:\\/\\/localhost:32771->https:\\/\\/brilliantcoding.com
(?i)/feed/(?!rss)->/feed/rss.xml
<atom:link href="http://localhost/2015/12/21/refactoring-to-strategy-pattern-in-javascript/feed/" rel="self" type="application/rss+xml" /><link>http://localhost:32771/2015/12/21/refactoring-to-strategy-pattern-in-javascript/</link>

<atom:link href="http://localhost/2015/12/21/refactoring-to-strategy-pattern-in-javascript/feed/" rel="self" type="application/rss+xml" /><link>https://brilliantcoding.com/2015/12/21/refactoring-to-strategy-pattern-in-javascript/</link>

HTML Replacements:1
<atom:link href="http://localhost/2015/12/21/refactoring-to-strategy-pattern-in-javascript/feed/" rel="self" type="application/rss+xml" /><link>https://brilliantcoding.com/2015/12/21/refactoring-to-strategy-pattern-in-javascript/</link>

<atom:link href="http://localhost/2015/12/21/refactoring-to-strategy-pattern-in-javascript/feed/rss.xml" rel="self" ty

success
